## Agent With Text to Cypher, Hybrid (Vector Index + GDS) Retrieval Via MCP

In [1]:
import os
import json
import getpass
from dotenv import load_dotenv
from expert_tools import ExpertTools

#get env setup
load_dotenv('podcast-gds.env', override=True)

if not os.environ.get('NEO4J_URI'):
    os.environ['NEO4J_URI'] = getpass.getpass('NEO4J_URI:\n')
if not os.environ.get('NEO4J_USERNAME'):
    os.environ['NEO4J_USERNAME'] = getpass.getpass('NEO4J_USERNAME:\n')
if not os.environ.get('NEO4J_PASSWORD'):
    os.environ['NEO4J_PASSWORD'] = getpass.getpass('NEO4J_PASSWORD:\n')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

#### Verify presence of VectorIndex, chunkIndex

In [2]:
# Verify vector index exists before creating the agent (no creation)

from neo4j import GraphDatabase

print("🔍 Verifying presence of vector index 'chunkIndex'...")
print(f"   Connecting to: {NEO4J_URI}")
print()

expert_temp = ExpertTools()

with expert_temp.driver.session() as session:
    result = session.run("SHOW VECTOR INDEXES")
    indexes = [record.data() for record in result]

    if not indexes:
        print("⚠️  No vector indexes found.")
    else:
        print(f"✅ Found {len(indexes)} vector index(es):\n")
        for idx in indexes:
            print(f"- {idx.get('name', 'N/A')} (state: {idx.get('state', 'N/A')})")

    chunk_index = [idx for idx in indexes if idx.get('name') == 'chunkIndex']
    if chunk_index:
        print(f"\n✅ Confirmed: 'chunkIndex' exists and is visible from this connection.")
        print(f"   State: {chunk_index[0].get('state', 'unknown')}")
    else:
        print(f"\n❌ 'chunkIndex' not found from this connection.")
        print("   If it exists elsewhere, check that:")
        print("   - You're connecting to the same database (URI and database name)")
        print("   - The user has privileges to see the index")

expert_temp.close()
print("\n✅ Check complete.")


🔍 Verifying presence of vector index 'chunkIndex'...
   Connecting to: neo4j+s://2236ba22.databases.neo4j.io

✅ Found 1 vector index(es):

- chunkIndex (state: ONLINE)

✅ Confirmed: 'chunkIndex' exists and is visible from this connection.
   State: ONLINE

✅ Check complete.


In [4]:
from AgentRunner import AgentRunner
import importlib
import expert_tools

# Reload the module to pick up the latest changes (fixes module caching issue)
importlib.reload(expert_tools)
from expert_tools import ExpertTools

# build adk agent with neo4j mcp
from neo4j_podcast_episode import PersonRel, PodcastEpisodeRel, EpisodeTopicRel, EpisodeChunkRel, TopicConceptRel, TopicTechnologyRel
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import Agent
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset, StdioConnectionParams, StdioServerParameters
from prompts import MENTAL_MODEL_AI_INSTRUCTION


expert = ExpertTools()
database_agent = Agent(
    name="graph_database_agent",
    # model="gemini-2.0-flash-exp",
    model=LiteLlm(model="openai/gpt-4.1-mini"),
    # model=LiteLlm(model="anthropic/claude-sonnet-4-20250514"),
    description="""
    Agent to access knowledge graph stored in graph database
    """,
    instruction=MENTAL_MODEL_AI_INSTRUCTION,
    tools=[expert.search_episodes_gds_by_question_tool,
           expert.find_episodes_by_people,
           expert.find_episodes_by_concept,
           expert.find_episodes_by_technology,
           expert.find_episodes_by_topic,
           McpToolset(
        connection_params=StdioConnectionParams(
            server_params=StdioServerParameters(
                command='uvx',
                args=['mcp-neo4j-cypher'],
                env={ k: os.environ[k] for k in ["NEO4J_URI","NEO4J_USERNAME","NEO4J_PASSWORD"] }
            )
        ),
        tool_filter=['get_neo4j_schema','read_neo4j_cypher']
    )]
)


db_agent_runner = AgentRunner(app_name='db_agent', user_id='Sang', agent=database_agent)
await db_agent_runner.start_session()

Session started successfully with ID: 7b49d662-d322-43ac-a7fe-2a82da2ec903


True

In [ ]:
from IPython.core.display import Markdown

In [6]:
user_question = "What is the most commnly discussed topic across the episodes?"
res = await db_agent_runner.run(user_question)

/Users/sangeethar/workspace/AI-Workspace/neo4j-employee-graph/neo4j-employee-graph/.venv/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


None id='call_2MKj39yy4Jz1DwVs8usV954Q' args={'query': 'MATCH (p:Person)-[r]->(e:Episode)-[:HAS_TOPIC]->(t:Topic) WHERE p.name = $personName RETURN t.name AS topic, count(*) AS freq ORDER BY freq DESC LIMIT 10'} name='read_neo4j_cypher' None


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


None None will_continue=None scheduling=None id='call_2MKj39yy4Jz1DwVs8usV954Q' name='read_neo4j_cypher' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='Neo4j Error: {neo4j_code: Neo.ClientError.Statement.ParameterMissing} {message: Expected parameter(s): personName} {gql_status: 42001} {gql_status_description: error: syntax error or access rule violation - invalid syntax}\nMATCH (p:Person)-[r]->(e:Episode)-[:HAS_TOPIC]->(t:Topic) WHERE p.name = $personName RETURN t.name AS topic, count(*) AS freq ORDER BY freq DESC LIMIT 10\n{}', annotations=None, meta=None)], structuredContent=None, isError=True)}


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


None id='call_PY8vrz6tmeBvsRvHTm47e2Dh' args={'query': "MATCH (p:Person)-[r]->(e:Episode)-[:HAS_TOPIC]->(t:Topic) WHERE p.name CONTAINS 'Sangeetha' RETURN t.name AS topic, count(*) AS freq ORDER BY freq DESC LIMIT 10"} name='read_neo4j_cypher' None
None None will_continue=None scheduling=None id='call_PY8vrz6tmeBvsRvHTm47e2Dh' name='read_neo4j_cypher' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='[{"topic": "Delayed View Semantics and Incremental Processing", "freq": 2}, {"topic": "Model Context Protocol (MCP): The Open Standard for Context-Aware AI", "freq": 2}, {"topic": "Duck Lake: The SQL-Backed Open Standard for Simplified Lakehouse Metadata", "freq": 2}, {"topic": "Embeddable Graph Databases with KuzuDB", "freq": 2}, {"topic": "BAML: Prompts as Structured Functions", "freq": 2}, {"topic": "Apache Iceberg Table Format and Data Lakes", "freq": 2}]', annotations=None, meta=None)], structuredContent=None, isError=False)}
The most commonly discu

In [5]:
user_question = "What has sangeetha been focussing on in the last one month. Give bulleted list of her focus areas and learning"
res = await db_agent_runner.run(user_question)

/Users/sangeethar/workspace/AI-Workspace/neo4j-employee-graph/neo4j-employee-graph/.venv/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


None id='call_iBnxOBV12PMAz3mqiO2rQSF8' args={'question': 'Sangeetha'} name='find_episodes_by_people' None


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


None None will_continue=None scheduling=None id='call_iBnxOBV12PMAz3mqiO2rQSF8' name='find_episodes_by_people' response={'result': '[\n  {\n    "person_name": "Sangeetha Ramadurai",\n    "relationship_type": "LEARNING_FROM",\n    "episode_name": "Prompts as Functions: The BAML Revolution in AI Engineering",\n    "episode_number": 2025040307,\n    "episode_link": "https://thedataexchange.media/baml-revolution-in-ai-engineering/",\n    "matched_term": "Sangeetha"\n  },\n  {\n    "person_name": "Sangeetha Ramadurai",\n    "relationship_type": "LISTENS_TO_EPISODE",\n    "episode_name": "Prompts as Functions: The BAML Revolution in AI Engineering",\n    "episode_number": 2025040307,\n    "episode_link": "https://thedataexchange.media/baml-revolution-in-ai-engineering/",\n    "matched_term": "Sangeetha"\n  },\n  {\n    "person_name": "Sangeetha Ramadurai",\n    "relationship_type": "LEARNING_FROM",\n    "episode_name": "Anthropic And Model Context Protocol (MCP) With David Soria Parra",\n   

In [ ]:
user_qusestion = "What are the top 3 AI related content Sangeetha is interested in based on the content covered in various episodes?" 
res = await db_agent_runner.run(user_qusestion)
display(Markdown(res))

In [ ]:
user_qusestion = "What are the top 20 technologies Sangeetha is interested in based on the content covered in various episodes?" 
res = await db_agent_runner.run(user_qusestion)

display(Markdown(res))

In [ ]:
user_qusestion = "What are the top 10 Databases that are covered across episodes?" 
res = await db_agent_runner.run(user_qusestion)

In [ ]:
user_question = "Gather and synthesize more detailed notes from relevant episodes that mention Iceberg or related lakehouse technologies to provide a broader landscape view"
res = await db_agent_runner.run(user_question)

In [ ]:
user_question = "Summarize discussions around Apache Iceberg"
res = await db_agent_runner.run(user_question)

In [ ]:
user_question = "What is Apache Iceberg and how does it work?"
res = await db_agent_runner.run(user_question)

In [ ]:
user_qusestion = "What are the top 2 episodes that cover Data Lake concepts" 
res = await db_agent_runner.run(user_qusestion)

In [ ]:
!mcp
!which mcp-neo4j-cypher

In [ ]:
!which uvx
!uvx --version
!uvx mcp-neo4j-cypher --help




In [ ]:
res = await db_agent_runner.run("what is Sangeetha Ramadurai learning now?")

In [ ]:
res = await db_agent_runner.run("What topics are discussed in the episodes?")

In [ ]:
res = await db_agent_runner.run("How many episodes do I have?")

In [ ]:
res = await db_agent_runner.run("Summarize snowflake capabilities discussed in the episodes?")

In [ ]:
# find_episodes_discussing_topics
res = await db_agent_runner.run("Find episodes discussing database and return one liner about the episode")

In [ ]:
res = await db_agent_runner.run("Find episodes with reference linking them to any database e.g. snowflake, kuzu, etc. and return one liner about the episode")

display(Markdown(res))

In [ ]:
res = await db_agent_runner.run("Find episodes where snowflake is referenced")

display(Markdown(res))

## Adding Expert Tools

### Tool - Ask free form question for Vector + graph response

In [ ]:
# Test the new Vector + Graph Search Tool
print("Testing Vector + Graph Search Tool...")
res = await db_agent_runner.run("How does KuzuDB improve performance in graph databases? Use the vector search tool to find relevant content.")


In [ ]:
# Test search methods with JSON formatting
import json
from expert_tools import ExpertTools

expert = ExpertTools()




In [ ]:
# Test the specific method you're using
print("=== Testing find_episodes_by_technology ===")
technology_res = expert.find_episodes_by_technology("Dynamic Tables, View")
print(technology_res)





In [ ]:
# Test other methods with JSON formatting
print("=== Testing find_episodes_by_topic ===")
topic_res = expert.find_episodes_by_topic("database, Graph")
print(topic_res)





In [ ]:
print("=== Testing find_episodes_by_people ===")
people_res = expert.find_episodes_by_people("Prashanth, Sangeetha")
print(people_res)





In [ ]:
print("=== Testing find_episodes_by_concept ===")
concept_res = expert.find_episodes_by_concept("graph database, database")
print(concept_res)



In [ ]:
# Clean up
expert.close()

## Standalone Expert Tools Testing

In [ ]:
from vector_graph_tool import VectorGraphSearchTool

# Direct usage of the Vector + Graph Search Tool
vector_tool = VectorGraphSearchTool()

# Test different types of questions
questions = [
    "How does KuzuDB improve performance in graph databases?",
    "What are the key features of Snowflake's dynamic tables?",
    "Who are the experts discussing data processing optimization?",
    "What technologies are mentioned in the episodes?"
]

for question in questions:
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print(f"{'='*60}")
    answer = vector_tool(question, top_k=3)
    print(answer)

# Clean up
vector_tool.close()


In [ ]:
await db_agent_runner.end_session()